In [6]:
from ultralytics import YOLO
import cv2
import mediapipe as mp

In [7]:
# Load pre-trained YOLOv8 model (nano is fastest, for person detection)
model = YOLO("yolov8n-pose.pt")

In [8]:
# Initialize pose and hands
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)


In [9]:
# Start webcam feed
cap = cv2.VideoCapture(0)  # Use 0 for default webcam

In [10]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process pose and hands
    pose_result = pose.process(rgb)
    hands_result = hands.process(rgb)

    # Draw pose landmarks (body stickman)
    if pose_result.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, pose_result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))

    # Draw hand landmarks (fingers)
    if hands_result.multi_hand_landmarks:
        for hand_landmarks in hands_result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2))

    cv2.imshow("Stickman with Fingers (Pose + Hand Tracking)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()